In [1]:
import simplekml
import teradata
import pandas as pd
from shapely.geometry import Polygon, Point

In [2]:
from matplotlib import pyplot as plt
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.ops import cascaded_union

import pandas as pd
import numpy as np

import re

In [3]:
#Teradata Connect
password = 'um6lrq9ecek'
udaExec = teradata.UdaExec (appName="HelloWorld", version="1.0",
                            logConsole=False)
session = udaExec.connect(method="odbc", system='td2800.corp.tele2.ru', username ="ZMITROCHENKO_EA", password=password,
                          driver="Teradata", charset='utf8')

### Load from Teradata

In [25]:
%%time
df = pd.read_sql(
"""select 
ne_id osm_id, level_name gks_pop, --cover_poly.ST_AsText() b,
geodata.ST_AsText() b,
'санкт петербург' reg_name
, 'ttt' name_tag
,1 WM_ID
from 
prd2_bds_v.detail_geodata_cover poly
where 1=1
and poly.ne_id --=592202362
in (

select distinct ne_id
from 
prd2_dds_v.network_element net_el
where 1=1
and (parent_ne_id = 193170012573 or ne_id = 193170012573)
--and ne_name like '%MO5940%'
and net_el.report_date = (select max(report_date) from prd2_dds_v.network_element )
)""", con=session)

Wall time: 6.32 s


In [5]:
%%time
df1 = pd.read_sql("""select * from uat_dm.zm_network_element_full_poly 
where 1=1
and bs_name = 'KE1082'""", con=session)

Wall time: 5.35 s


In [6]:
df1.head()

,BS_NAME,NE_ID,STECHNOLOGY,LEVEL_NAME,EFFECTIVE_TO,COVER_POLY,REGION_ID
0,KE1082,4.617911e+10,4G,indoor,2999-12-31,POLYGON ((87.897091104280648 52.75930218109225...,29.0
1,KE1082,4.617911e+10,2G,deep indoor,2999-12-31,POLYGON ((87.91687825701085 52.749265707631572...,29.0
2,KE1082,4.617911e+10,2G,incar,2999-12-31,"POLYGON ((87.776533961619052 52.6855647389648,...",29.0
3,KE1082,4.617911e+10,2G,outdoor,2999-12-31,POLYGON ((87.91687825701085 52.749265707631572...,29.0
4,KE1082,4.617911e+10,2G,deep indoor,2999-12-31,POLYGON ((87.91687825701085 52.749265707631572...,29.0


## Preprocessing

In [4]:
def to_gpd_polygones(string_):
    from re import findall
    re_list = [float(x) for x in findall(r'[0-9]+.[0-9]+',string_)]
    return Polygon(tuple(zip(re_list[0::2], re_list[1::2])))
    #return (list(zip(re_list[0::2], re_list[1::2])))

In [7]:
def to_kml_polygones(string_):
    from re import findall
    re_list = [float(x) for x in findall(r'[0-9]+.[0-9]+',string_)]
    #return Polygon(tuple(zip(re_list[0::2], re_list[1::2])))
    return (list(zip(re_list[0::2], re_list[1::2])))

In [8]:
df1['GPD_POLYGONES'] = df1['COVER_POLY'].map(to_gpd_polygones)

In [9]:
gdf = gpd.GeoDataFrame(df1[['NE_ID','STECHNOLOGY','LEVEL_NAME','GPD_POLYGONES']]) #'GEOTIFF_FILE_ID' ,
#gdf = gpd.GeoDataFrame(df[['osm_id','reg_name','gks_pop','name_tag', 'WM_ID', 'GPD_POLYGONES']])

In [10]:
gdf.head(1)

,NE_ID,STECHNOLOGY,LEVEL_NAME,GPD_POLYGONES
0,4.617911e+10,4G,indoor,"POLYGON ((87.89709110428065 52.75930218109225,..."


In [11]:
gdf.set_geometry('GPD_POLYGONES')
gdf['geometry']=gdf['GPD_POLYGONES']

## Create KML

In [12]:
kml = simplekml.Kml()

for level in gdf.loc[:,u"LEVEL_NAME"].unique():
    print(level)
    fol = kml.newfolder(name=level)
    
    gdf_s = gdf.loc[gdf[u"LEVEL_NAME"]==level,:].iloc[:,0:len(gdf)]
    
    for index, row in gdf_s.iterrows():
        NE_ID = row['NE_ID']
        GEOTIFF_FILE_ID = row['GEOTIFF_FILE_ID']
        STECHNOLOGY = row['STECHNOLOGY']
        LEVEL_NAME = row['LEVEL_NAME']
        POLYGON = to_kml_polygones(str(row['geometry']))
        #POLYGON = str(row['geometry'])

        descr = 'GEOTIFF_FILE_ID = '+str(GEOTIFF_FILE_ID)+('\n')\
                +'STECHNOLOGY = '+str(STECHNOLOGY)+('\n')+'LEVEL_NAME = '+str(LEVEL_NAME)
 

        
        pol = fol.newpolygon(name=str(NE_ID), description=descr,
                             outerboundaryis=POLYGON)
        pol.style.linestyle.color = simplekml.Color.yellow
        pol.style.linestyle.width = 1
        pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.yellow)

indoor


KeyError: 'GEOTIFF_FILE_ID'

In [314]:
kml.save("contains_all.kml")